In [1]:
import pickle as pkl
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import random
from sklearn.preprocessing import MinMaxScaler
from statistics import mean

In [2]:
train_df = pd.read_pickle('pdb_train.p')
test_df = pd.read_pickle('pdb_test.p')

In [3]:
train_df

,pdb_id,lig_id,seq,smiles,receptor_features,ligand_xyz,ligand_xyz_2d,ligand_bonds
0,1ict,T44,CPLMVKVLDAVRGSPAINVAVHVFRKAADDTWEPFASGKTSESGEL...,N[C@@H](Cc1cc(I)c(Oc2cc(I)c(O)c(I)c2)c(I)c1)C(...,{'rigidgroups_gt_frames': [[[[ 9.7112149e-01 -...,"[(-7.698, -45.225, 30.127), (-6.968, -45.819, ...","[(-5.063699296693056, 1.3648647122225173, 0.0)...","[(4, 6), (6, 7), (7, 11), (11, 32), (32, 36), ..."
1,1v1j,FA3,PRSLANAPIMILNGPNLNLLGQRQPEIYGSDTLADVEALCVKAAAA...,O=C(O)[C@]1(O)C=C(F)[C@@H](O)[C@H](O)C1,{'rigidgroups_gt_frames': [[[[-0.09479982 -0.8...,"[(30.735, 53.717, 58.584), (nan, nan, nan), (2...","[(2.3944077361532012, 1.7261843001158517, 0.0)...","[(25, 15), (15, 21), (21, 23), (23, 17), (17, ..."
2,1iyo,CEF,SVQQQLEALEKSSGGRLGVALINTADNSQILYRADERFAMCSTSKV...,C=C1CS[C@H]([C@@H](C=O)NC(=O)C(=NOC)c2csc(N)n2...,{'rigidgroups_gt_frames': [[[[ -0.9025636 -0...,"[(-1.789, -44.887, 63.321), (nan, nan, nan), (...","[(-6.8702167036559185, -0.4250690130944643, 0....","[(5, 4), (4, 2), (2, 0), (0, 40), (40, 42), (4..."
3,5s7o,XJJ,QRTVARDITLLECVGKGRYGEVWRGSWQGENVAVKIFSSRDEKSWF...,O=C1CNCCN1,{'rigidgroups_gt_frames': [[[[-8.2274151e-01 ...,"[(6.496, 37.594, -35.347), (nan, nan, nan), (7...","[(2.5714285714285716, 1.9032394707859828e-16, ...","[(0, 2), (2, 4), (4, 5), (5, 6), (5, 7), (7, 8..."
4,6vuc,RLS,SNPPPPETSNPNKPKRQTNQLQYLLRVVLKTLWKHQFAWPFQQPVD...,CN1C[C@@H](c2ccc(S(=O)(=O)N3CCCCC3)cc2)CC1=O,{'rigidgroups_gt_frames': [[[[ 9.9730277e-01 ...,"[(-10.435, -5.41, 12.772), (-11.641, -5.312, 1...","[(-7.0025400830107705, 1.6316778784387103, 0.0...","[(0, 35), (35, 34), (34, 4), (4, 6), (6, 8), (..."
...,...,...,...,...,...,...,...,...
34272,6vxq,RQS,YGSWEIDPKDLTFLKELGTGQFGVVKYGKWRGQYDVAIKMIKEGSM...,O=C(NCc1ccc(-c2ncnc3[nH]ccc23)cc1)c1ccccc1,{'rigidgroups_gt_frames': [[[[ 0.80936086 0.3...,"[(13.2, -7.469, 8.261), (nan, nan, nan), (13.2...","[(4.867117161770941, 2.4807207768480257, 0.0),...","[(17, 4), (4, 23), (23, 13), (13, 10), (10, 9)..."
34273,7apv,9F2,FEGQMAEYPTISIDRFDRENLRARAYFLSHCHKDHMKGLRAPTLKR...,CON=C(C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(CSc3nc(=...,{'rigidgroups_gt_frames': [[[[ 0.5408227 0.3...,"[(-6.856, 7.889, 6.967), (-8.078, 7.93, 7.637)...","[(-7.324411940502822, -4.625058374149916, 0.0)...","[(55, 54), (54, 53), (53, 31), (31, 33), (33, ..."
34274,2aad,2GP,ACDYTCGSNCYSSSDVSTAQAAGYKLHEDGETVGSNSYPKKYNNYE...,Nc1nc2c(ncn2[C@@H]2O[C@H](CO)[C@@H](O)[C@H]2OP...,{'rigidgroups_gt_frames': [[[[-0.8275972 0.1...,"[(47.148, 27.661, -3.213), (46.21, 27.806, -2....","[(-4.836839894151689, 2.9444781193120653, 0.0)...","[(27, 30), (30, 33), (33, 35), (35, 18), (18, ..."
34275,6gda,EVB,AEFKAGSAKKGATLFKTRCLQCHTVEKGGPHKVGPNLHGIFGRHSG...,O=S(=O)(O)c1cc2c(O)c(c1)Cc1cc(S(=O)(=O)O)cc(c1...,{'rigidgroups_gt_frames': [[[[ 5.9049261e-01 ...,"[(-33.768, -9.798, 15.743), (nan, nan, nan), (...","[(3.557228638561006, -10.773567442510728, 0.0)...","[(12, 13), (13, 15), (15, 192), (192, 48), (48..."


In [4]:
#random.seed(42)

#train_df = train_df.sample(n=1000, random_state=42)
train_df = train_df.sample(n=1000)
#test_df = test_df.sample(n=1000, random_state=42)
test_df = test_df.sample(n=1000)

In [5]:
f = open('pdb_id_binds_to_lig_id.pckl', 'rb')
pdb_id_binds_to_lig_id = pkl.load(f)
f.close()


In [6]:
def generate_negative_samples(df, negative_samples_per_positive_sample, pdb_id_binds_to_lig_id):
    negative_samples = []
    n_negative_samples = negative_samples_per_positive_sample*len(df)
    
    while len(negative_samples) < n_negative_samples:
        idx1, idx2 = np.random.choice(range(len(df)), 2, replace=False)
        pdb_id = list(df['pdb_id'])[idx1]
        lig_id = list(df['lig_id'])[idx2]
        
        if lig_id not in pdb_id_binds_to_lig_id[pdb_id]:
            negative_sample = df.iloc[idx1].copy()
            negative_sample['ligand_xyz'] = df.iloc[idx2]['ligand_xyz']
            negative_sample['ligand_xyz_2d'] = df.iloc[idx2]['ligand_xyz_2d']
            negative_sample['ligand_bonds'] = df.iloc[idx2]['ligand_bonds']
            negative_sample['smiles'] = df.iloc[idx2]['smiles']
            negative_samples.append(negative_sample)
            
    return pd.DataFrame(negative_samples)

In [7]:
negative_samples_per_positive_sample = 2

In [8]:
n_positive_samples = len(train_df)
n_negative_samples = negative_samples_per_positive_sample * n_positive_samples

negative_samples = generate_negative_samples(train_df, negative_samples_per_positive_sample, pdb_id_binds_to_lig_id)
train_df = pd.concat([train_df, negative_samples], ignore_index=True)
y_train = pd.concat([pd.Series(1, index=range(n_positive_samples)), 
                     pd.Series(0, index=range(n_negative_samples))])

In [9]:
n_positive_samples = len(test_df)
n_negative_samples = negative_samples_per_positive_sample * n_positive_samples

negative_samples = generate_negative_samples(test_df, negative_samples_per_positive_sample, pdb_id_binds_to_lig_id)
test_df = pd.concat([test_df, negative_samples], ignore_index=True)
y_test = pd.concat([pd.Series(1, index=range(n_positive_samples)),
                    pd.Series(0, index=range(n_negative_samples))])

In [10]:
seq_vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(1, 3))
seq_train = seq_vectorizer.fit_transform(train_df['seq'])
seq_test  = seq_vectorizer.transform(test_df['seq'])


In [11]:
print(seq_train)

  (0, 96)	0.02788335309812557
  (0, 3906)	0.024024429901115845
  (0, 6676)	0.020257423893133352
  (0, 5950)	0.024934880075605077
  (0, 5054)	0.03189842544507267
  (0, 3285)	0.030156198839325743
  (0, 3631)	0.026435279838287223
  (0, 1480)	0.022842702066718967
  (0, 4834)	0.02496858401424989
  (0, 7598)	0.02660551479631104
  (0, 5994)	0.03022968505363155
  (0, 6777)	0.029462396116154595
  (0, 3370)	0.02772877337809618
  (0, 6649)	0.023434480312922928
  (0, 330)	0.024801728760585295
  (0, 879)	0.023222685742556266
  (0, 1781)	0.02705097047611193
  (0, 1389)	0.02700507322158055
  (0, 932)	0.024867974830247212
  (0, 6531)	0.024901344237103547
  (0, 5513)	0.025988208025398127
  (0, 2006)	0.029016095350990805
  (0, 7456)	0.02705097047611193
  (0, 4273)	0.022892142890070764
  (0, 5402)	0.02192333087300136
  :	:
  (2999, 2143)	0.014875458796123129
  (2999, 1386)	0.011665338774983483
  (2999, 6983)	0.02416129847669136
  (2999, 3378)	0.02321926160902859
  (2999, 4910)	0.050098109504633566
  (299

In [12]:
smiles_vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(1, 3))
smiles_train = smiles_vectorizer.fit_transform(train_df['smiles'])
smiles_test  = smiles_vectorizer.transform(test_df['smiles'])

In [13]:
print(smiles_train)

  (0, 128)	0.0898653853304996
  (0, 290)	0.10396981244955572
  (0, 372)	0.11816008324067968
  (0, 768)	0.11878828374859055
  (0, 726)	0.06122819641657821
  (0, 35)	0.05694405409386167
  (0, 390)	0.09376666392078176
  (0, 135)	0.06078966025344677
  (0, 364)	0.10582348589781036
  (0, 766)	0.07959562290901524
  (0, 831)	0.1392869539447543
  (0, 117)	0.08560849573929441
  (0, 946)	0.09202186978373973
  (0, 849)	0.13632423498155674
  (0, 43)	0.06499500897347128
  (0, 939)	0.07070270805754271
  (0, 848)	0.07491151408941606
  (0, 839)	0.13014778189228185
  (0, 399)	0.06799898589066028
  (0, 774)	0.1221172876723753
  (0, 999)	0.17770256031422874
  (0, 64)	0.07196175365079101
  (0, 718)	0.059043989765048634
  (0, 318)	0.07980473050379842
  (0, 967)	0.08895091255967605
  :	:
  (2999, 715)	0.07128583734331935
  (2999, 1038)	0.06201899473732817
  (2999, 826)	0.08753420215010327
  (2999, 837)	0.07845229465426565
  (2999, 829)	0.034332371193711686
  (2999, 721)	0.08388161344448265
  (2999, 942)	0.05

In [14]:
X_train = pd.concat([pd.DataFrame(seq_train.toarray()), pd.DataFrame(smiles_train.toarray())], axis=1) 
X_test = pd.concat([pd.DataFrame(seq_test.toarray()), pd.DataFrame(smiles_test.toarray())], axis=1)


In [15]:
def k_fold_cross_validation_scores(clf,x,y,k):

    kf = KFold(n_splits=k, shuffle=True, random_state=21)

    Tot_acc=[]
    Tot_pre=[]
    Tot_rec=[]
    Tot_f1=[]

    i=0

    for train, test in kf.split(x):

        i=i+1
        print("\nFold %d" %(i))
        
        x_train, x_test = x.iloc[train],  x.iloc[test]
        y_train, y_test = y.iloc[train], y.iloc[test]
        y_pred = clf.fit(x_train, y_train).predict(x_test)

        acc=accuracy_score(y_test, y_pred)
        pre=precision_score(y_test, y_pred)
        rec=recall_score(y_test, y_pred)
        f1=f1_score(y_test, y_pred)
        
        Tot_acc.append(acc)
        Tot_pre.append(pre)
        Tot_rec.append(rec)
        Tot_f1.append(f1)

    print ("\nAverage Accuracy: %2.3f" % (mean(Tot_acc)))
    print ("\nAverage Precision: %2.3f" % (mean(Tot_pre)))
    print ("\nAverage Recall: %2.3f" % (mean(Tot_rec)))
    print ("\nAverage F1-score: %2.3f" % (mean(Tot_f1)))

In [16]:
x= pd.concat([X_train,X_test])
y= pd.concat([y_train,y_test])

print("SVM:")
svm = SVC(kernel='rbf', C=1, gamma='scale')
k_fold_cross_validation_scores(clf=svm, x=x, y=y, k=5)

print("\nLogistic Regression:")
lr = LogisticRegression(C=1, solver='liblinear', random_state=42)
k_fold_cross_validation_scores(clf=lr, x=x, y=y, k=5)

print("\nRandom Forest:")
rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
k_fold_cross_validation_scores(clf=rf, x=x, y=y, k=5)

SVM:

Fold 1

Fold 2


C:\Users\13404\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Fold 3

Fold 4

Fold 5


C:\Users\13404\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Average Accuracy: 0.667

Average Precision: 0.600

Average Recall: 0.002

Average F1-score: 0.004

Logistic Regression:

Fold 1

Fold 2

Fold 3

Fold 4

Fold 5

Average Accuracy: 0.659

Average Precision: 0.075

Average Recall: 0.002

Average F1-score: 0.004

Random Forest:

Fold 1

Fold 2


C:\Users\13404\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Fold 3


C:\Users\13404\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Fold 4

Fold 5

Average Accuracy: 0.666

Average Precision: 0.000

Average Recall: 0.000

Average F1-score: 0.000


C:\Users\13404\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [32]:
svm = SVC(kernel='rbf', C=1, gamma='scale')
svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'SVM:')
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')  
print(f'Recall: {recall:.4f}')
print(f'F1 score: {f1:.4f}')

SVM:
Accuracy: 0.6673
Precision: 1.0000
Recall: 0.0020
F1 score: 0.0040


In [33]:
lr = LogisticRegression(C=1, solver='liblinear', random_state=42)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Logistic Regression:') 
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 score: {f1:.4f}')

Logistic Regression:
Accuracy: 0.6647
Precision: 0.2500
Recall: 0.0030
F1 score: 0.0059


In [34]:
rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)  
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Random Forest:')
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 score: {f1:.4f}')

Random Forest:
Accuracy: 0.6667
Precision: 0.0000
Recall: 0.0000
F1 score: 0.0000


C:\Users\13404\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
